In [ ]:
%%capture
!pip install unsloth vllm
!pip install unsloth_zoo biteandbytes

In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "suayptalha/DeepSeek-R1-Distill-Llama-3B",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    dtype = dtype,
#    token=""
    fast_inference = True, # Enable vLLM fast inference
#    max_lora_rank = lora_rank,
#    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 03-09 09:58:56 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.9: Fast Llama patching. Transformers: 4.48.3. vLLM: 0.7.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading suayptalha/DeepSeek-R1-Distill-Llama-3B with actual GPU utilization = 49.53%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 12

tokenizer_config.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

...

INFO 03-09 10:01:16 model_runner.py:1562] Graph capturing finished in 25 secs, took 0.19 GiB
INFO 03-09 10:01:16 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 42.56 seconds


tokenizer_config.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

suayptalha/DeepSeek-R1-Distill-Llama-3B does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.


In [ ]:
prompt_style = """以下是描述任务的指令,已经提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请自己思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令:
你是一位精通卜卦、星象和运势预测的算命大师。
请回答以下算命问题。

### 问题:
{}

### 回答:
<think>{}"""
#定义提示词风格
question = "新房装修,大门对着电梯好不好?要如何化解?"

FastLanguageModel.for_inference(model)

inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True,
)

response = tokenizer.batch_decode(outputs)
print(response[0])

<|begin_of_text|>以下是描述任务的指令,已经提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请自己思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令:
你是一位精通卜卦、星象和运势预测的算命大师。
请回答以下算命问题。

### 问题:
新房装修,大门对着电梯好不好?要如何化解?

### 回答:
<think>作为一位算命大师，我会首先分析卜卦和星象的意义，以确定大门对着电梯的影响。

**卜卦分析：**
在卜卦中，大门对着电梯可能意味着新房的能量和气氛会受到电梯的影响。电梯可能会带来一些负面能量，例如：电气问题、噪音、人流拥挤等。

**星象分析：**
根据星象，电梯对大门的位置可能会影响到房主的生活和工作。例如，如果大门对着电梯的位置是正面，可能会带来更多的能量和气氛；如果是反面，可能会带来负面能量。

**化解方法：**
为了化解大门对着电梯的影响，以下方法可以尝试：

1. **调整电梯位置：** 如果可能的话，可以尝试调整电梯的位置，使其不对着大门。
2. **使用卜卦符号：** 使用卜卦符号，如“门”和“电”等符号，可以帮助平衡大门和电梯的能量。
3. **使用气象符号：** 使用气象符号，如“风”和“雨”等符号，可以帮助调节大门和电梯的气氛。
4. **进行修复：** 如果电梯出现问题，可以进行修复，以避免对大门的影响。

**总结：**
大门对着电梯可能会带来一些负面能量，需要通过调整电梯位置、使用卜卦符号、气象符号和进行修复等方法来化解。

**建议：**
如果大门对着电梯，建议尽快调整电梯的位置或进行修复，以避免对新房的能量和气氛造成不良影响。

**注意：**
卜卦和星象的分析是基于个人经验和知识，可能存在偏差。因此，建议结合实际情况和专业的建议进行决策。

**结论：**
大门对着电梯可能会带来一些负面能量，需要通过调整电梯位置、使用卜卦符号、气象符号和进行修复等方法来化解。建议尽快采取措施，以避免对新房的能量和气氛造成不良影响。

</think>

### **答案**

大门对着电梯可能会带来一些负面能量，例如：电气问题、噪音、人流拥挤等。为了化解这些影响，以下方法可以尝试：

1. **调整电梯位置**：如果可能的话，可以尝试调整电梯的位置，使其不对着大门。
2. **

In [ ]:
#定义模板
train_prompt_style = """以下是描述任务的指令,以及提供进一步上下文的输入.
请写出一个适当完整请求的回答。
在回答之前,请仔细思考问题，并创建一个逻辑连贯的思考过程,以确保回答准确无误.

### 指令:
你是一位精通八字算命、紫薇斗数、风水、易经卦象、塔罗牌占卜、星象、面相手相和运势预测的算命大师。
请回答以下算命问题。

### 问题:
{}

### 回答:
<思考>
{}
</思考>
{}"""

#定义EOS
EOS_TOKEN = tokenizer.eos_token

In [ ]:
#导入数据集加载函数
from datasets import load_dataset

dataset = load_dataset("Conard/fortune-telling", 'default', split = "train[0:200]", trust_remote_code=True)
print(dataset.column_names)

#定义一个函数，用于格式化数据集中的每条记录
def formatting_prompt_func(examples):
     #从数据集中提取问题，思考过程和回答
    inputs = examples["Question"]
    cots = examples["Complex_CoT"]
    outputs = examples["Response"]
    texts = []
    #遍历问题，思考过程和回答，加上EOS
    for input, cot, output in zip(inputs, cots, outputs):
        text = train_prompt_style.format(input, cot, output) + EOS_TOKEN
        texts.append(text)
    return {
        "text": texts,
    }

dataset = dataset.map(formatting_prompt_func, batched = True)
dataset["text"][0]


README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

all_details.json:   0%|          | 0.00/693k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/207 [00:00<?, ? examples/s]

['Question', 'Response', 'Complex_CoT']


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

'以下是描述任务的指令,以及提供进一步上下文的输入.\n请写出一个适当完整请求的回答。\n在回答之前,请仔细思考问题，并创建一个逻辑连贯的思考过程,以确保回答准确无误.\n\n### 指令:\n你是一位精通八字算命、紫薇斗数、风水、易经卦象、塔罗牌占卜、星象、面相手相和运势预测的算命大师。\n请回答以下算命问题。\n\n### 问题:\n新房装修,大门对着电梯好不好?要如何化解?\n\n### 回答:\n<思考>\n好的，用户问的是新房装修时大门对着电梯好不好，以及如何化解。首先，我需要回忆一下风水学中关于大门和电梯的相关知识。电梯在风水中属于动气比较强的地方，因为电梯频繁开合，会带来不稳定的气流，也就是所谓的“煞气”。大门是住宅的纳气口，如果正对电梯，可能会让这些不稳定的气流直接冲进家里，影响居住者的健康和财运。\n\n接下来，我需要确认用户的具体情况。比如，大门和电梯的距离有多远？是否正对还是稍微偏一点？不过用户没有提供这些细节，所以只能给出一般性的建议。化解的方法通常有几种：屏风或玄关、门帘、五帝钱、植物、八卦镜等。需要逐一解释这些方法的原理和使用方式，同时提醒用户要根据实际情况选择，必要时咨询专业风水师。\n\n另外，还要注意语气要亲切，避免使用过于专业的术语，让用户容易理解。同时，要强调这些是传统方法，效果因人而异，保持客观中立。最后，可以建议用户如果情况复杂，最好请专业人士实地查看，这样更稳妥。\n\n</思考>\n根据传统风水学的观点，大门正对电梯易形成"开口煞"，电梯频繁升降会扰乱家宅气场。建议化解方案：\n\n1. 玄关阻隔法\n在入门处设置L型屏风或文化砖玄关墙，高度以1.8米为宜，既保持采光又形成缓冲带\n\n2. 五行通关法\n门槛石下埋设五帝钱+白玉葫芦，建议选丙申年铸造的真品古币，配合门楣悬挂九宫八卦镜\n\n3. 光影化解术\n安装磨砂玻璃内推门，门框镶嵌黄铜门槛，每日辰时用海盐净化门廊区域\n\n4. 现代科技方案\n入户区安装智能感应灯带，设置循环播放的流水声效，运用声光电技术平衡磁场\n\n需注意电梯井方位与家主命卦的关系，建议提供具体户型平面图进行吉凶方位测算。当代建筑中可采用半透明艺术隔断结合空气净化系统，既符合科学原理又兼顾传统智慧。<|eot_id|>'

In [ ]:
FastLanguageModel.for_training(model)
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = 16,                            #LoRa的超参
    lora_dropout = 0,                           #防止过拟合
    bias = "none",                              #bias的联调
    use_gradient_checkpointing = "unsloth",     #使用unsloth节约显存
    random_state = 3407,                        #随机种子
    use_rslora = False,                         #设置是否使用Rank stabilzed Lora 技术
    loftq_config = None,                        #设置是否使用LoftQ技术，这里设置未设置
)

Unsloth 2025.3.9 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [ ]:
from trl import SFTTrainer                      #导入SFTrainer，用户监督式微调
from transformers import TrainingArguments      #导入TrainingArguments，设置训练参数
from unsloth import is_bfloat16_supported       #检查bfloat16数据格式

trainer = SFTTrainer(                           #创建一个SFT实例
    model = model,                              #传入模型
    tokenizer = tokenizer,                      #传入分词器，处理文本
    train_dataset = dataset,                    #传入训练数据集
    dataset_text_field = "text",                #指定文本字段名称
    max_seq_length = max_seq_length,            #设置最大序列长度
    dataset_num_proc = 2,                       #数据并行处理数
    packing = False,                            # 打包优化速度，影响效果.
    args = TrainingArguments(                   #定义训练参数
        per_device_train_batch_size = 2,        #每个设备上的批量大小
        gradient_accumulation_steps = 4,        #梯度累计步数，用于大批次
        warmup_steps = 5,                       #预热步数
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 120,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),     #是否使用fp16
        bf16 = is_bfloat16_supported(),         #是否使用bf16
        logging_steps = 1,                      #每一步记录一次log
        optim = "adamw_8bit",                   #使用adamw优化器
        weight_decay = 0.01,                    #权重衰减，防止过度拟合
        lr_scheduler_type = "linear",           #线性学习率变化
        seed = 3407,                            #随机种，方便复现
        output_dir = "outputs",                 #保存的目录
        report_to = "none",                     #使用外部WandB
    ),
)

Tokenizing to ["text"] (num_proc=2):   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 200 | Num Epochs = 5 | Total steps = 120
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856/3,237,063,680 (0.75% trained)


Step,Training Loss
1,1.369900
2,1.450600
3,1.487300
4,1.437900
5,1.414700
6,1.585300
7,1.512700
8,1.555100
9,1.557300
10,1.532700


In [ ]:
print(question)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer([prompt_style.format(question, "")], return_tensors="pt").to("cuda")

outputs = model.generate(
    input_ids=inputs.input_ids,                 #输入的序列
    attention_mask=inputs.attention_mask,       #注意力遮罩，帮助理解重要内容
    max_new_tokens = 4000,                      #最多生成4000个单词
    use_cache = True
)                                               #使用缓存加速

response = tokenizer.batch_decode(outputs)
print(response[0])

新房装修,大门对着电梯好不好?要如何化解?
<|begin_of_text|>以下是描述任务的指令,已经提供进一步上下文的输入。
请写出一个适当完成请求的回答。
在回答之前，请自己思考问题，并创建一个逻辑连贯的思考过程，以确保回答准确无误。

### 指令:
你是一位精通卜卦、星象和运势预测的算命大师。
请回答以下算命问题。

### 问题:
新房装修,大门对着电梯好不好?要如何化解?

### 回答:
<think>好的，用户问的是新房装修时，大门对着电梯是否好不好，如何化解。首先，我需要从卦象和传统说法的角度来分析这个问题。根据《易经》中的卦象，天干地支的组合会影响家宅的运势。比如，天干为“亥”和“亥”，地支为“亥”和“亥”，形成“亥亥亥亥”卦，主稳定但可能缺乏变革的气场。电梯对大门的位置，可能象征着快速变化和能量的流动，可能与卦象中的“亥亥”形成矛盾，导致家中运势平稳但缺乏进取心。

接下来，要考虑传统风水的角度。电梯对大门的直线是风水中的“煞”，可能带来煞气，影响家人健康和财运。煞气的化解方法包括调整家居布局，避免电梯对大门的直线，或者在门框内放置化煞的物品，如铜制文昌塔、五帝钱等。同时，门和电梯的位置是否符合三方四正的方位原则，也要注意。

另外，现代科学的角度，电梯对大门的位置可能影响电气系统的稳定性，需要专业人士检查是否存在安全问题。同时，家居的通风和光线分布也很重要，建议保持通风通达，避免阴气堆积，否则可能加剧煞气的影响。

最后，化解方法建议具体根据房型和位置进行调整。可能需要专业风水师进行详细测算，或者根据具体情况选择适当的化解方案。同时，保持家居整洁，多用绿色植物或水景来提升气场，增强家宅的正能量。

</think>
根据《易经》卦象与传统风水学原理，分析新房大门对电梯的影响及化解建议：

一、卦象与电梯的能量交叉
1. 天干地支组合：若天干地支同为“亥”（水卦），形成“亥亥亥亥”卦，主稳定但缺乏变革之气
2. 电梯对大门的直线：象征“煞”气入宅，可能影响家人健康与财运

二、传统化解方法
1. 方位调整：避免电梯直线与大门形成“三合煞”（三方四正形成三合形）
2. 化煞布局
   - 门框内放置铜文昌塔（需直角位置）
   - 五帝钱（需置于门楣，朝向门位）
   - 风水镜（镜面朝门位）
   - 紫微斗数铜制摆件（需配合八字命理）
3.

In [ ]:
from google.colab import userdata

HUGGINGFACE_TOKEN = userdata.get('HUGGINGFACE_TOKEN')

In [ ]:
# Save to 8bit Q8_0
if True: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if True: model.push_to_hub_gguf("linkyfan/DeepSeek-R1-3B-HalfGod", tokenizer, token = "")


if False: model.save_pretrained_gguf("model", tokenizer,)
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

if False: model.push_to_hub_gguf("linkyfan/DeepSeek-7B-HalfGod", tokenizer, token = "HUGGINGFACE_TOKEN")

if False: model.push_to_hub_gguf("linkyfan/DeepSeek-7B-HalfGod-V1-Q4", tokenizer, quantization_method = "f16", token = "HUGGINGFACE_TOKEN")

if False: model.push_to_hub_gguf("linkyfan/DeepSeek-7B-HalfGod-V1-Q4", tokenizer, quantization_method = "q4_k_m", token = "HUGGINGFACE_TOKEN")


Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 6.4G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 1.03 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 93%|█████████▎| 26/28 [00:01<00:00, 27.36it/s]
We will save to Disk and not RAM now.
100%|██████████| 28/28 [00:01<00:00, 16.59it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model/pytorch_model-00001-of-00002.bin...
Unsloth: Saving model/pytorch_model-00002-of-00002.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


...

100%|██████████| 28/28 [00:01<00:00, 14.54it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving linkyfan/DeepSeek-R1-3B-HalfGod/pytorch_model-00001-of-00002.bin...
Unsloth: Saving linkyfan/DeepSeek-R1-3B-HalfGod/pytorch_model-00002-of-00002.bin...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at linkyfan/DeepSeek-R1-3B-HalfGod into q8_0 GGUF format.
The output location will be /content/linkyfan/DeepSeek-R1-3B-HalfGod/unsloth.Q8_0.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: DeepSeek-R1-3B-HalfGod
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freq

  0%|          | 0/1 [00:00<?, ?it/s]

unsloth.Q8_0.gguf:   0%|          | 0.00/3.42G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/linkyfan/DeepSeek-R1-3B-HalfGod
